In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
class Image_Custom_Augmentation:

    def __init__(self, SP_intensity, RO_Key, Br_intensity, HV_Key = False, HE_Key = False):
        # Salt and Pepper Intensity
        self.SP_intensity = SP_intensity 
        # Brightness Intensity
        self.Br_intensity = Br_intensity 
        # Vertical/Horizontal Flip Key
        self.HV_Key = HV_Key 
        # Rotate Key
        self.RO_Key = RO_Key 
        # Histogram Equalization Key
        self.HE_Key = HE_Key 
        
        
    def Salt_n_Pepper(self, image_path, output_dir):
        image = cv2.imread(image_path)
        clean_label = os.path.splitext(os.path.basename(image_path))[0]
        # Create a Salt&Pepper filter
        height, width, _ = image.shape

        # Generate random noise mask
        salt_mask = np.random.rand(height, width) < self.SP_intensity
        pepper_mask = np.random.rand(height, width) < self.SP_intensity
        
        # Apply salt noise
        image[salt_mask] = [255,255,255]  # Set pixel to white (salt)
        image[pepper_mask] = [0,0,0]  # Set pixel to black (pepper)

        # Save the modified image to the output path
        custom_name = f"{clean_label}"+"_SP_"+".jpg"
        output_path = os.path.join(output_dir, custom_name)
        cv2.imwrite(output_path, image)
        
        # Reset
        del salt_mask, pepper_mask, image, clean_label, output_path, custom_name
            
            
    def Histogram_Equalization(self, image_path, output_dir):
        image = cv2.imread(image_path)
        clean_label = os.path.splitext(os.path.basename(image_path))[0]

        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply histogram equalization
        equalized_image = cv2.equalizeHist(gray_image)

        # Save the modified image to the output path
        custom_name = f"{clean_label}"+"_HE_"+".jpg"
        output_path = os.path.join(output_dir, custom_name)
        cv2.imwrite(output_path, equalized_image)

        # Reset
        del equalized_image, gray_image, image, clean_label, custom_name, output_path
     
    
    def Rotate(self, image_path, output_dir):
        image = cv2.imread(image_path)
        clean_label = os.path.splitext(os.path.basename(image_path))[0]
        
        # Apply Rotation
        height, width = image.shape[:2]
        centerX, centerY = (width // 2, height // 2)
        
        "Get the rotation Matrix and apply it to the image"
        M_1 = cv2.getRotationMatrix2D((centerX, centerY), self.RO_Key, 1.0)
        M_2 = cv2.getRotationMatrix2D((centerX, centerY), -self.RO_Key, 1.0)
        ccw_rotated_image = cv2.warpAffine(image, M_1, (width, height))
        cw_rotated_image = cv2.warpAffine(image, M_2, (width, height))
    
        # Save the modified image to the output path
        custom_name_1 = f"{clean_label}"+"_CCWR_"+".jpg"
        custom_name_2 = f"{clean_label}"+"_CWR_"+".jpg"
        output_path_1 = os.path.join(output_dir, custom_name_1)
        output_path_2 = os.path.join(output_dir, custom_name_2)
        cv2.imwrite(output_path_1, ccw_rotated_image)
        cv2.imwrite(output_path_2, cw_rotated_image)
        
        # Reset
        del ccw_rotated_image, cw_rotated_image, custom_name_1, custom_name_2, output_path_1, output_path_2, image, clean_label
    
    
    def Brightness(self, image_path, output_dir):
        image = cv2.imread(image_path)
        clean_label = os.path.splitext(os.path.basename(image_path))[0]
        # Create a 2D plate of same values to Add/Subtract from the initial image
        plate = np.ones(image.shape, dtype="uint8") * (self.Br_intensity)
        # Two different filters (Br/Da)
        brighter_img = cv2.add(image, plate)
        darker_img = cv2.subtract(image, plate)
        
        # Save the modified images to the output path
        custom_name_1 = f"{clean_label}"+"_BR_"+".jpg"
        custom_name_2 = f"{clean_label}"+"_DA_"+".jpg"
        output_path_1 = os.path.join(output_dir, custom_name_1)
        output_path_2 = os.path.join(output_dir, custom_name_2)
        cv2.imwrite(output_path_1, brighter_img)
        cv2.imwrite(output_path_2, darker_img)
        
        # Reset
        del brighter_img, darker_img, image, clean_label, custom_name_1, custom_name_2, output_path_1, output_path_2
        
        
    def Flip_HV(self, image_path, output_dir):
        image = cv2.imread(image_path)
        clean_label = os.path.splitext(os.path.basename(image_path))[0]
        # Flip the image 
        V_flipped = cv2.flip(image, 0)
        H_flipped = cv2.flip(image, 1)
        
        # Save the modified images to the output path
        custom_name_1 = f"{clean_label}"+"_VF_"+".jpg"
        custom_name_2 = f"{clean_label}"+"_HF_"+".jpg"
        output_path_1 = os.path.join(output_dir, custom_name_1)
        output_path_2 = os.path.join(output_dir, custom_name_2)
        cv2.imwrite(output_path_1, V_flipped)
        cv2.imwrite(output_path_2, H_flipped)
        
        # Reset
        del V_flipped, H_flipped, image, clean_label, custom_name_1, custom_name_2, output_path_1, output_path_2
        
        
    def Generate_Data(self, input_path, output_path):
        for index in tqdm(os.listdir(input_path)):
            image_path = os.path.join(input_path, index)
            # Switching between functions
            if self.HV_Key:
                self.Flip_HV(image_path, output_dir=output_path)
            
            if self.Br_intensity:
                self.Brightness(image_path, output_dir=output_path)
           
            if self.HE_Key:
                self.Histogram_Equalization(image_path, output_dir=output_path)
                
            if self.RO_Key:
                self.Rotate(image_path, output_dir=output_path)
               
            if self.SP_intensity:
                self.Salt_n_Pepper(image_path, output_dir=output_path)
                
            else:
                print("Error! No functionality has been called.")
        

In [3]:
My_data = Image_Custom_Augmentation(SP_intensity=0.035,
                                    RO_Key=60,
                                    Br_intensity=40,
                                    HV_Key = True,
                                    HE_Key = True)

In [4]:
input_path = "C:\\Users\\Eurus\\Desktop\\Train_Data\\images"
output_path = "C:\\Users\\Eurus\\Desktop\\Train_Data\\images_augmented"

My_data.Generate_Data(input_path,output_path)

100%|██████████████████████████████████████████████████████████████████████████████| 1271/1271 [01:19<00:00, 15.89it/s]


In [ ]:
# del My_data